In [5]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline ,HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import (
    FewShotPromptTemplate
)
import pickle
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)

def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

def load_and_vectorize(csv_path, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    # CSV 파일을 불러와 데이터프레임 생성
    train_df = pd.read_csv(csv_path)
    train_df.drop('SAMPLE_ID', axis=1, inplace=True)
    trainset = train_df.to_dict(orient='records')
    print("Loaded Fewshot Set:", trainset[:3])
    
    # 벡터화할 텍스트 생성
    to_vectorize = ["\n\n".join(example.values()) for example in trainset]
    
    # 벡터화 및 FAISS DB 생성
    fewshow_vectordb = FAISS.from_texts(to_vectorize, embedding=get_embedding(), metadatas=trainset)
    
    # FAISS DB 저장
    save_faiss_db(fewshow_vectordb, db_path)
    
    return fewshow_vectordb

def save_faiss_db(db, db_path):
    db.save_local(db_path)
    with open(db_path + '_metadata.pkl', 'wb') as f:
        pickle.dump(db, f)

def load_faiss_db(db_path):
    db = FAISS.load_local(db_path, embeddings=get_embedding(), allow_dangerous_deserialization=True)
    with open(db_path + '_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    return db, metadata

def load_chunks_make_docdb(pdf_directory, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    print("Loading PDF files from:", pdf_directory)
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf').replace('\\', '/'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)
    
    # 분할된 텍스트를 벡터로 변환하여 FAISS DB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    print("Done.", len(chunks), "chunks")
    
    print("Creating FAISS DB")
    # FAISS DB 생성 및 저장
    db = FAISS.from_documents(chunks, embedding=get_embedding())
    save_faiss_db(db, db_path)
    print("Done.")
    
    return db

def make_dict(dir='train.csv'):
    df = pd.read_csv(dir)
    df.drop('SAMPLE_ID', axis=1, inplace=True)
    
    return df.to_dict(orient='records')


In [6]:
fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')


train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
test_retriever = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

train_dict = make_dict('train.csv')
test_dict = make_dict('test.csv')

Loading FAISS DB from: ./fewshot_faiss_db
Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


In [13]:
def make_fewshot_prompt(fewshot_vectordb, k = 3):
    # Semantic Similarity Example Selector 설정
    example_prompt = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")

    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_vectordb,
        k=k,
    )

    # FewShotPromptTemplate 생성
    fewshot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        suffix="Question: {input}",
        input_variables=["input"],
    )
    return fewshot_prompt

def make_fewshot_string(fewshot_prompt, train_retriever, buff):
    ex_qa = fewshot_prompt.invoke({"input": buff['Question']}).to_string()
    fewshot_list = ex_qa.split('\n\n')[:-1]
    for i, entry in enumerate(fewshot_list):
        question = entry.split('\n')[0]
        question = question.replace('Question: ', '')
        retrieved_docs = train_retriever.invoke(question)
        entry += retrieved_docs[0].page_content
        num = "Example {}\n".format(i+1)
        fewshot_list[i] = num + entry + '\n\n'
    return str(fewshot_list)

fewshot_prompt = make_fewshot_prompt(fewshot_db)
i = 24
print(test_dict[i]['Question'])
print(test_dict[i]['Source'])
print('==============================')


ex_qa = fewshot_prompt.invoke({"input": test_dict[i]['Question']}).to_string()
fewshot_list = ex_qa.split('\n\n')[:-1]
for i, entry in enumerate(fewshot_list):
    question = entry.split('\n')[0]
    question = question.replace('Question: ', '')
    retrieved_docs = train_retriever.invoke(question)
    entry += retrieved_docs[0].page_content
    num = "Example {}\n".format(i+1)
    fewshot_list[i] = num + entry + '\n\n'
print(str(fewshot_list))

에너지바우처 사업의 주요 수혜자는 누구인가요?
산업통상자원부_에너지바우처


In [4]:
# for i, entry in enumerate(fewshot_list):
#     question = entry.split('\n')[0]
#     question = question.replace('Question: ', '')
#     retrieved_docs = train_retriever.invoke(question)
#     entry += retrieved_docs[0].page_content
#     num = "Example {}\n".format(i+1)
#     fewshot_list[i] = num + entry + '\n\n'

    
# for i in fewshot_list:
#     print(i)
#     print('==============================')


In [5]:
# fewshot_str = str(fewshot_list)
# fewshot_str

In [6]:
# for i in range(len(test_dict)):
#     print(test_dict[i]['Question'])
#     print(test_dict[i]['Source'])
    
#     fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
#     print(fewshot_str)
#     print('==============================')

In [7]:


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def setup_llm_pipeline(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)
    hf = HuggingFacePipeline(pipeline=pipe)
    return hf

def run():
    fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
    fewshot_prompt = make_fewshot_prompt(fewshot_db)
    
    train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
    train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})
    
    test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
    test_retriver = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})
    
    train_dict = make_dict('train.csv')
    test_dict = make_dict('test.csv')
    
    llm = setup_llm_pipeline("beomi/Llama-3-Open-Ko-8B")
    results =[]
    for i in tqdm(range(len(test_dict))):
        print(test_dict[i]['Question'])
        fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
        # print(fewshot_str)
        full_template = """
        너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
        너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
        반드시 context를 참고하여 답변을 작성해야해.
        예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.
        """+f"""
        이건 질문과 관련된 예시야.
        {fewshot_str}
        """+"""
        
        이 context를 참고하여 아래 질문에 대한 답변을 작성해주세요.
        {context}

        질문은 이것이다.
        {input}
        """
        prompt = PromptTemplate.from_template(full_template)
        qa_chain = (
        {
            "context": test_retriver | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )
        
        results.append({
            "Question": test_dict[i]['Question'],
            "Answer": qa_chain.invoke(test_dict[i]['Question']),
            "Source": test_dict[i]['Source']
        
            })
        print("================================================")
        print("Questions: ",results[-1]['Question'])
        print("Answer: ",results[-1]['Answer'])
        #print(results[-1]['Source'])
        
    # 제출용 샘플 파일 로드
    submit_df = pd.read_csv("./sample_submission.csv")
    # 생성된 답변을 제출 DataFrame에 추가
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

    # 결과를 CSV 파일로 저장
    submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)

In [8]:
def run():
    fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
    fewshot_prompt = make_fewshot_prompt(fewshot_db)
    
    train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
    train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})
    
    test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
    test_retriver = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})
    
    train_dict = make_dict('train.csv')
    test_dict = make_dict('test.csv')
    
    llm = setup_llm_pipeline("beomi/Llama-3-Open-Ko-8B")
    results =[]
    for i in tqdm(range(len(test_dict))):
        print(test_dict[i]['Question'])
        fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
        # print(fewshot_str)
        full_template = """
        너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
        너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
        반드시 context를 참고하여 답변을 작성해야해.
        예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.
        """+f"""
        이건 질문과 관련된 예시야.
        {fewshot_str}
        """+"""
        
        이 context를 참고하여 아래 질문에 대한 답변을 작성해주세요.
        {context}

        질문은 이것이다.
        {input}
        """
        prompt = PromptTemplate.from_template(full_template)
        qa_chain = (
        {
            "context": test_retriver | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )
        
        results.append({
            "Question": test_dict[i]['Question'],
            "Answer": qa_chain.invoke(test_dict[i]['Question']),
            "Source": test_dict[i]['Source']
        
            })
        print("================================================")
        print("Questions: ",results[-1]['Question'])
        print("Answer: ",results[-1]['Answer'])
        #print(results[-1]['Source'])
        
    # 제출용 샘플 파일 로드
    submit_df = pd.read_csv("./sample_submission.csv")
    # 생성된 답변을 제출 DataFrame에 추가
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

    # 결과를 CSV 파일로 저장
    submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)
run()

Loading FAISS DB from: ./fewshot_faiss_db
Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]